---
# 1. Reading in the NCEP wind data (.nc files)
This section will load in the wind data from two .nc files (u and v components). Once loaded, we need to determine how the x,y,z dimentions of the data matrix correspond to the lat, lon, time dimensions--this is done easily from the metadata contained within the file. Nothing else is required to complete this section.

Wind data is available from a variety of sources with variable resolution (spatial and temporal) and difficulty in integrating with this script (easy to import .nc to custom binary formats).

*  ~0.3 degree, 3hr, .nc format (N. America only) -- [NARR](https://www.esrl.noaa.gov/psd/data/gridded/data.narr.monolevel.html)
* 2.5 degree, 6hr, .nc format -- [NCEP](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.surface.html)
* 0.25 degree, 6hr, .nc format (inc. cloud cover) -- [NCSS](https://www.ncei.noaa.gov/thredds/ncss/uv/6h_agg_rt/Preliminary_Aggregation_best.ncd/dataset.html)
* 0.25 degree, hourly, .D binary format -- [NAVGEM](ftp://ftp.hycom.org/datasets/force/NAVGEM/navgem1.4_0.281c/1hourly/)

In [ ]:
source('source.r')

In [3]:
## If there are saved datasets, then run this to load them

load('./RStates/wind.lat.rdata')
load('./RStates/wind.lon.rdata')
load('./RStates/wind.ncep.rdata')
load('./RStates/time.actual.rdata')

In [2]:
get.info = function(path) {
    u.nc = nc_open(path, write=FALSE)
    ## Use the file metadata to populate ranges and coordinate mappings
    u.nc
    nc_close(u.nc)
}

read.wind = function(path.u, path.v, time.bounds, real.time = TRUE) {
    ## Open the nc files (adapt as needed)
    u.nc = nc_open(path.u, write=FALSE)
    v.nc = nc_open(path.v, write=FALSE)

    ## Load the data into u,v, lat, lon
    wind.u = ncvar_get(u.nc, varid = 'uwnd')
    wind.v = ncvar_get(v.nc, varid = 'vwnd')
    wind.lat = ncvar_get(u.nc, varid = 'lat')
    wind.lon = ncvar_get(u.nc, varid = 'lon')
    
    ## Clsoe the files
    nc_close(u.nc)
    nc_close(v.nc)

    dims = dim(wind.u)
    
    if (real.time) {
        ## Calculate the actual times
        time.actual = seq(time.bounds[1]*3600, time.bounds[2]*3600, length.out = dims[3])
        time.actual = as.POSIXct(time.actual, origin='1800-01-01 00:00:00')  # orgin is in the file metadata above
    }

    ## Check on time format/range
    summary(time.actual)
}


In [ ]:
read.wind(path.u = 'C:/Users/Tom/Downloads/uwnd.10m.2017.nc',
          path.v = 'C:/Users/Tom/Downloads/vwnd.10m.2017.nc',
          time.bounds = c(1902192, 1906533))

### Flatten the lat/lon dimensions
Instead of sticking with the (i, j, t) -> (lat,lon, time) coordinate mapping setup, let's flatten it out to make a simpler structure so that we have:

`(i, t) - > (lat, lon, time)`   --- convert a 3-D matrix into a 2-D one (easier to manage)

Therefore we can simply the x-y grid to i = 1,2,3,4,5,6,7,8,..., x*y

In [ ]:
## Calculate wind speed & Flatten
wind.ncep = array(NA, dim=c(dim(wind.u)[1] * dim(wind.u)[2], dim(wind.u)[3]))
for (i in 1:dim(wind.u)[3]) {
    wind.ncep[,i] = sqrt(as.numeric(wind.u[,,i]^2 + wind.v[,,i]^2))
}

## Flatten the lat and lon arrays as well
wind.lat = as.numeric(wind.lat) # x,y -> i
wind.lom = as.numeric(wind.lon) # x,y -> i

### Trim the data
Take the time now to trim the data  to a more reasonable domain to speed up later computations (looping is expensive).

In [ ]:
## First, check the domains of the data
print(paste('The minimum and maximum latitude in the NCEP data is', min(wind.lat), max(wind.lat)))
print(paste('The minimum and maximum longitude in the NCEP data is', min(wind.lon), max(wind.lon)))
print(paste('The minimum date in the NCEP data is', min(time.actual)))
print(paste('The minimum date in the NCEP data is', max(time.actual)))

In [ ]:
## First set some generic bounds
lats = c(30, 40)
lons = c(-126, -118)

## Calcualte which entries should be kept (latitude)
keep.lat = wind.lat > lats[1] & wind.lat < lats[2] # Which col to keep
keep.lon = wind.lon > lons[1] & wind.lon < lons[2] # Which row to keep
keepers = keep.lat & keep.lon

## update all required variables
before = length(wind.ncep)

wind.lat = wind.lat[keepers]
wind.lon = wind.lon[keepers]
wind.ncep = wind.ncep[keepers, ]

print(paste("This cut removed", 100-length(wind.ncep)/before*100, '% of the data.'))

In [4]:
## Save wind data here -- can reload directly

wind = list(ncep = wind.ncep, lat = wind.lat, lon = wind.lon, time = time.actual)
save(wind, file = 'RStates/wind.rdata')